In [1]:
from __future__ import unicode_literals, print_function, division
import pandas as pd
import os
from io import open
import glob
import unicodedata
import string
import torch
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
def tokenize(sentence, word_set):
  words = []
  start = 0
  while start < len(sentence):
    find = True
    for stop in range(len(sentence), start, -1):
      if sentence[start: stop].lower() in word_set:
        words.append(sentence[start: stop])
        start = stop
        break
      if stop == start + 1:
        find = False
        break
    if not find:
      start += 1
  return words

## get vocabulary set

In [4]:
data = pd.read_excel('2018_tagged_data.xlsx')

In [5]:
data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Bug缺少模块,下载/传文件,操作问题,操作问题(gps),抱怨/退费,要求/改进功能,商务,无响应,Unnamed: 31,Unnamed: 32
0,2018081000006,山西水塔醋业营销有限责任公司6.0,3.517021e+09,山西水塔醋业营销有限责任公司6.0,山西省,太原市,2018-08-10 09:01:55,8001成凤伟,服务异常,红圈通,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,考勤 不,技术 紧急 处理
1,2018040400034,售前咨询,1.805793e+10,售前咨询,浙江省,金华市,2018-04-04 09:53:02,8009范香香,售前咨询红圈通,商机,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,网络 科技 有限 公司 咨询 软件 ，七八十 人，外出 定位 轨迹 ，费用,转
2,2018011900041,北京华戎凯威植保生物科技有限公司,1.885504e+10,bjhrkwzbsw,安徽省,滁州市,2018-01-19 11:05:23,8013苏永辉,使用咨询手机4.0,定位,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,华为 青春,已调 手机 设置
3,2018030700059,定州市佳谊家电销售有限公司6.0,1.348380e+10,定州市佳谊家电销售有限公司6.0,河北省,保定市,2018-03-07 10:46:07,8013苏永辉,使用咨询红圈通,定位,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,魅族 定位 离线,已调 手机 设置
4,2018030600052,成都齐力红食品有限责任公司,1.361808e+10,cdqlh,四川省,成都市,2018-03-06 10:34:51,8009范香香,售后咨询红圈营销,找业务员,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,对接 红圈 定位 问题,转


In [8]:
word_set = set()
for i in range(5001):
  situation = str(data['Unnamed: 31'][i]).replace('，',' ').split() + str(data['Unnamed: 32'][i]).replace('，',' ').split()
  for w in situation:
    word_set.add(w)

print(len(word_set))

2400


In [0]:
data = pd.read_excel('2018 payroll.xlsx', header = None)

In [0]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,2018081000006,山西水塔醋业营销有限责任公司6.0,3.517021e+09,山西水塔醋业营销有限责任公司6.0,山西省,太原市,2018-08-10 09:01:55,8001成凤伟,服务异常,红圈通,服务中断,考勤不了 18636980333,技术在紧急处理,产品问题,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,考勤 不,技术 紧急 处理
1,2018040400034,售前咨询,1.805793e+10,售前咨询,浙江省,金华市,2018-04-04 09:53:02,8009范香香,售前咨询红圈通,商机,网络推广,金华淘够网络科技有限公司 郭先生 咨询软件 ，七八十人，外出时要定位轨迹 ，了解费用,转,客户问题,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,网络 科技 有限 公司 咨询 软件 ， 七八十 人 ， 外出 定位 轨迹 ， 费用,转
2,2018011900041,北京华戎凯威植保生物科技有限公司,1.885504e+10,bjhrkwzbsw,安徽省,滁州市,2018-01-19 11:05:23,8013苏永辉,使用咨询手机4.0,定位,手机设置错误,华为青春版定位灰点,已调手机设置,客户问题,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,华为 青春,已 调 手机 设置
3,2018030700059,定州市佳谊家电销售有限公司6.0,1.348380e+10,定州市佳谊家电销售有限公司6.0,河北省,保定市,2018-03-07 10:46:07,8013苏永辉,使用咨询红圈通,定位,手机设置错误,魅族note6定位离线,已调手机设置,客户问题,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,魅族 定位 离线,已 调 手机 设置
4,2018030600052,成都齐力红食品有限责任公司,1.361808e+10,cdqlh,四川省,成都市,2018-03-06 10:34:51,8009范香香,售后咨询红圈营销,找业务员,联系不上/无联系方式,成都齐力红食品有限责任公司 找对接人员 红圈定位问题 宁女士 13618079712,转,客户问题,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,对接 红圈 定位 问题,转


## use bigram

In [0]:
def load_embeddings(path):
    embedding_map = {}
    with open(path) as f:
        next(f)
        for line in f:
            try:
                pieces = line.rstrip().split()
                embedding_map[pieces[0]] = [float(weight) for weight in pieces[1:]]
            except:
                pass
    return embedding_map
d = load_embeddings('chinese_words.txt')

In [0]:
print(len(d.keys()))
print(type(d.keys()))
word_set = set(d.keys())

364171
<class 'dict_keys'>


In [0]:
print(len(data))

36951


## split new rows

In [0]:
word_set = set([w.lower() for w in word_set])
print(len(word_set))

362553


In [0]:
data = pd.read_excel('2018_tagged_data.xlsx')
problems = data['Unnamed: 31']
situation = data['Unnamed: 32']
for i in range(len(data)):
  sent1 = str(data['Unnamed: 31'][i])
  sent2 = str(data['Unnamed: 32'][i])
  #print(i)
  #print(sent1)
  word1 = tokenize(sent1, word_set)
  #print(' '.join(word1))
  word2 = tokenize(sent2, word_set)
  #print(sent2)
  #print(' '.join(word2))
  #print()
  problems[i] = ' '.join(word1)
  situation[i] = ' '.join(word2)

data['Unnamed: 31'] = problems
data['Unnamed: 32'] = situation
data.to_excel('2018 payroll.xlsx', header=False, index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
